In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

In [2]:
class MDN(tf.keras.layers.Layer):
    def __init__(self, l, k, **kwargs):
        self.l = l
        self.k = k
        self.mu = layers.Dense(k * l, name='mu')
        self.sigma = layers.Dense(k * l, activation='')
        super(MDN, self).__init__(**kwargs)